In [1]:
import pandas as pd
import os
import datetime
import glob
import numpy as np
os.getcwd()

'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3'

In [2]:
df_data=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_2019-07-20/dfrfm_final_details_wemail_ziplabel_StoreQuad.csv",
              dtype=str,usecols=['customer_id_hashed','email_address_hash','frmindex','HML_Group','zipcodegroup','segment_2019Q3'])
df_data.shape,df_data['customer_id_hashed'].nunique()

((18936098, 6), 18936098)

In [5]:
df_total_count=df_data.groupby(["frmindex",'HML_Group','zipcodegroup'])['customer_id_hashed','email_address_hash'].nunique().reset_index()
df_total_count.shape

(30, 5)

# 2019Q3 shopping label

In [6]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [8]:
Q2_dates=[datetime.date(2019,5,5),datetime.date(2019,8,3)]
Q3_dates=[datetime.date(2019,8,4),datetime.date(2019,11,2)]

weekly_2019_daily_df=list(recursive_file_gen("/home/jian/BigLots/"))

weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2018" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2017" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2016" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2016" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "Historical" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "hist_daily_data_itemlevel_decompressed" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if ".txt" in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "aily" in x]
weekly_2019_daily_df=pd.DataFrame({"file_path":weekly_2019_daily_df})
weekly_2019_daily_df['week_end_dt']=weekly_2019_daily_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])

weekly_2019_daily_df['week_end_dt']=weekly_2019_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
print(weekly_2019_daily_df.shape)
weekly_2019_daily_df.head(2)

weekly_2019_daily_Q2=weekly_2019_daily_df[(weekly_2019_daily_df['week_end_dt']>Q2_dates[0]) & (weekly_2019_daily_df['week_end_dt']<=Q2_dates[1])]
weekly_2019_daily_Q3=weekly_2019_daily_df[(weekly_2019_daily_df['week_end_dt']>Q3_dates[0]) & (weekly_2019_daily_df['week_end_dt']<=Q3_dates[1])]

print(weekly_2019_daily_Q2.shape,weekly_2019_daily_Q3.shape)

(38, 2)
(13, 2) (7, 2)


In [11]:
print(datetime.datetime.now())

shoppers_2019Q2=pd.DataFrame()
shoppers_2019Q3=pd.DataFrame()


for file in weekly_2019_daily_Q2['file_path'].tolist():
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    shoppers_2019Q2=shoppers_2019Q2.append(df)
    print(file,datetime.datetime.now())
shoppers_2019Q2=shoppers_2019Q2.sort_values(['customer_id_hashed','transaction_dt'],ascending=[True,False])
shoppers_2019Q2=shoppers_2019Q2.drop_duplicates(['customer_id_hashed'])
shoppers_2019Q2['Q2_shopping_flag']="Q2_Shopped"
shoppers_2019Q2=shoppers_2019Q2.rename(columns={"transaction_dt":"last_shop_date_Q2"})
print(datetime.datetime.now())

for file in weekly_2019_daily_Q3['file_path'].tolist():
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    shoppers_2019Q3=shoppers_2019Q3.append(df)
    print(file,datetime.datetime.now())
shoppers_2019Q3=shoppers_2019Q3.sort_values(['customer_id_hashed','transaction_dt'],ascending=[True,False])
shoppers_2019Q3=shoppers_2019Q3.drop_duplicates(['customer_id_hashed'])
shoppers_2019Q3['Q3_shopping_flag']="Q3_Shopped"
shoppers_2019Q3=shoppers_2019Q3.rename(columns={"transaction_dt":"last_shop_date_Q3"})
print(datetime.datetime.now())


2019-09-25 12:55:37.930723
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-11/MediaStormDailySales20190514-121223-405.txt 2019-09-25 12:55:57.298677
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-18/MediaStormDailySales20190521-111232-391.txt 2019-09-25 12:56:16.769467
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-25/MediaStormDailySales20190528-111211-863.txt 2019-09-25 12:57:30.394750
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-01/MediaStormDailySales20190604-111936-950.txt 2019-09-25 12:59:55.398250
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-08/MediaStormDailySales20190611-122013-442.txt 2019-09-25 13:01:56.799297
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-15/MediaStormDailySales20190618-112500-846.txt 2019-09-25 13:04:47.222420
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-22/MediaStormDailySales20190625-111927-257.txt 2019-09-25 13:08:14.073869
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-29/MediaStormDailySales20190702-113523-

In [19]:
df_data=pd.merge(df_data,shoppers_2019Q2,on="customer_id_hashed",how="outer")
df_data=pd.merge(df_data,shoppers_2019Q3,on="customer_id_hashed",how="outer")

In [22]:
df_data.head(2)

,frmindex,customer_id_hashed,email_address_hash,zipcodegroup,HML_Group,segment_2019Q3,last_shop_date_Q2,Q2_shopping_flag,last_shop_date_Q3,Q3_shopping_flag
0,D01,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,4836bfc632a2a5987d47fa6bf0637c21c4466a0e99ddee...,P,H,Quadrant I_P_H_2019Q3,2019-08-03,Q2_Shopped,2019-09-21,Q3_Shopped
1,D01,bfd2e98c3c7cccf2f0d432f8d02d951479311150e4a540...,bff5e20f383626e52c464341ad29b3b4e161b8b35de99d...,P,H,Quadrant I_P_H_2019Q3,2019-08-03,Q2_Shopped,2019-09-21,Q3_Shopped


In [23]:
df_data_M=df_data[df_data['HML_Group']=="M"]
df_data_D57=df_data[df_data['frmindex'].isin(['D05','D06','D07'])]
print(df_data_M.shape,df_data_D57.shape)

(5680830, 10) (5680830, 10)


In [27]:
set(df_data_M.index==df_data_D57.index)

{True}

In [29]:
df_data_D57['last_shop_date_Q2']=df_data_D57['last_shop_date_Q2'].fillna("NoShop")
df_data_D57['Q2_shopping_flag']=df_data_D57['Q2_shopping_flag'].fillna("NoShop")
df_data_D57['last_shop_date_Q3']=df_data_D57['last_shop_date_Q3'].fillna("NoShop")
df_data_D57['Q3_shopping_flag']=df_data_D57['Q3_shopping_flag'].fillna("NoShop")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [40]:
output_summary_D57=df_data_D57.groupby(['HML_Group','frmindex','zipcodegroup','Q3_shopping_flag','Q2_shopping_flag'])['customer_id_hashed','email_address_hash'].nunique().reset_index()
output_summary_D57.shape



(36, 7)

In [ ]:
write_folder='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/Q3_M_Shopping_in_Q3/'
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

In [35]:
df_data_D57.to_csv(write_folder+"df_data_D57.csv",index=False)
df_data.to_csv(write_folder+"df_data.csv",index=False)


In [41]:
df_data['HML_Group']=df_data['HML_Group'].fillna("nan")
df_data['frmindex']=df_data['frmindex'].fillna("nan")

output_summary_count_all=df_data.groupby(['HML_Group','frmindex','zipcodegroup','Q3_shopping_flag','Q2_shopping_flag'])['customer_id_hashed'].nunique().to_frame().reset_index()

In [43]:
writer=pd.ExcelWriter(write_folder+"BL_Q3_M_Shopping_in_Q3_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_summary_D57.to_excel(writer,"count_M_only",index=False)
output_summary_count_all.to_excel(writer,"count_all_ids",index=False)
writer.save()

# To Spencer

In [48]:
df_data_D57.head(2)

,frmindex,customer_id_hashed,email_address_hash,zipcodegroup,HML_Group,segment_2019Q3,last_shop_date_Q2,Q2_shopping_flag,last_shop_date_Q3,Q3_shopping_flag
7574440,D05,e9e88d85e01a9ebab175190a1e4610e76f1ae23f1b379c...,6c047e08b4bb5b528fb7bd6b49202db21e98bdeeec7dd2...,S,M,Quadrant IV_S_M_2019Q3,NoShop,NoShop,NoShop,NoShop
7574441,D05,1f93fe197af50fa268e789bb66aeef93fffe2e4de36883...,ae1d4dc68a0d7e1426df95294959ada496bdd05c72d354...,P,M,Quadrant I_P_M_2019Q3,NoShop,NoShop,NoShop,NoShop


In [50]:
all_uploaded_audience=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/upload_files/*.csv")
df_all_M_contol_test=pd.DataFrame()
for file in all_uploaded_audience:
    if "_M_" in file:
        df=pd.read_csv(file,dtype=str)
        df['segment']=df['segment'].apply(lambda x: x[0])
        df_all_M_contol_test=df_all_M_contol_test.append(df)
df_all_M_contol_test=df_all_M_contol_test.rename(columns={"segment":"T_or_C_Q3RFM"})
df_all_M_contol_test.shape,df_all_M_contol_test['customer_id_hashed'].nunique()

((5009142, 3), 5009142)

In [52]:
df_all_M_contol_test['T_or_C_Q3RFM'].unique()

array(['T', 'C'], dtype=object)

In [61]:
print(datetime.datetime.now())
df_data_D57_PS_NoShopped_Yet=df_data_D57[(df_data_D57['Q3_shopping_flag']=="NoShop") & (df_data_D57['zipcodegroup']!="T")]
print(df_data_D57_PS_NoShopped_Yet.shape)


df_data_D57_PS_NoShopped_Yet=pd.merge(df_data_D57_PS_NoShopped_Yet,df_all_M_contol_test,on=['customer_id_hashed','email_address_hash'],how="left")
print(df_data_D57_PS_NoShopped_Yet.shape)


2019-09-25 14:53:31.294546
(4105718, 10)
(4105718, 11)


In [62]:
df_data_D57_PS_NoShopped_Yet['T_or_C_Q3RFM'].unique()

array(['T', 'C'], dtype=object)

In [64]:
df_data_D57_PS_NoShopped_Yet[pd.isnull(df_data_D57_PS_NoShopped_Yet['T_or_C_Q3RFM'])]

,frmindex,customer_id_hashed,email_address_hash,zipcodegroup,HML_Group,segment_2019Q3,last_shop_date_Q2,Q2_shopping_flag,last_shop_date_Q3,Q3_shopping_flag,T_or_C_Q3RFM


In [65]:
df_data_D57_PS_NoShopped_Yet.shape

(4105718, 11)

In [67]:
df_data_D57_PS_NoShopped_Yet.to_csv(write_folder+"BL_2019Q3_audience_M_PS_no_shopped_yet_7_weeks_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [68]:
import paramiko

os.listdir(write_folder)

['df_data_D57.csv',
 'df_data.csv',
 'BL_Q3_M_Shopping_in_Q3_JL_2019-09-25.xlsx',
 'BL_2019Q3_audience_M_PS_no_shopped_yet_7_weeks_JL_2019-09-25.csv']

In [69]:
local_path=write_folder+"BL_2019Q3_audience_M_PS_no_shopped_yet_7_weeks_JL_2019-09-25.csv"
print(local_path)

/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/Q3_M_Shopping_in_Q3/BL_2019Q3_audience_M_PS_no_shopped_yet_7_weeks_JL_2019-09-25.csv


In [71]:
remote_folder="/mnt/drv5/lr_biglots_data/Q3_2019_not_shopped_yet/"
remote_path=remote_folder+os.path.basename(local_path)
print(remote_path)

/mnt/drv5/lr_biglots_data/Q3_2019_not_shopped_yet/BL_2019Q3_audience_M_PS_no_shopped_yet_7_weeks_JL_2019-09-25.csv


In [72]:
host = "64.237.51.251" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "jian@juba2017" #hard-coded
username = "jian" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

sftp.put(local_path,remote_path)
sftp.close()
transport.close()


In [74]:
df_data_D57_PS_NoShopped_Yet.shape

(4105718, 11)